# Traveling Salesman Problem

## Model Initialization

In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

# initialize model object
mymodel = glp.Solver('Traveling Salesman', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

In [2]:
# import cost matrix as data frame
cost = [[ 0.0,  5.0, 15.0, 10.0], # cost from node 0
        [ 5.0,  0.0, 12.0, 20.0],  # node 1
        [15.0, 12.0,  0.0,  6.0],  # node 2
        [10.0, 20.0,  6.0,  0.0]]  # node 3

# coerce data type = integer / numeric?
# Probably need a way to create the dummny data from a regular cost matrix

# row = starting node, column = ending node (i to j)

In [3]:
cost

[[0.0, 5.0, 15.0, 10.0],
 [5.0, 0.0, 12.0, 20.0],
 [15.0, 12.0, 0.0, 6.0],
 [10.0, 20.0, 6.0, 0.0]]

In [4]:
# model parameters
N = len(cost) # number of nodes in data frame (including the dummy node)
M = N + 1000.0 # arbitrarily large number

In [5]:
N

4

In [6]:
M

1004.0

## Decision Variables

In [7]:
use_arc = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 
for i in range(N):
    for j in range(N):
        use_arc[i][j] = mymodel.IntVar(0, 1, str(i) + "." + str(j))
use_arc

[[0.0, 0.1, 0.2, 0.3],
 [1.0, 1.1, 1.2, 1.3],
 [2.0, 2.1, 2.2, 2.3],
 [3.0, 3.1, 3.2, 3.3]]

In [8]:
# create position variable (lambda / u)
pos = list(range(N))
pos[0] = mymodel.IntVar(0, 0, 'p_0')
for i in range(1,N):
    pos[i] = mymodel.IntVar(1, N-1, "p_" + str(i))
pos

[p_0, p_1, p_2, p_3]

## Objective

In [9]:
# create objective function
shortest_route = mymodel.Objective()
shortest_route.SetMinimization()
for i in range(N):
    for j in range(N):
        shortest_route.SetCoefficient(use_arc[i][j], cost[i][j])

## Constraints

### Arc constraints

In [10]:
# Balance - Out
# 2.1 + 2.2 + 2.3 + 2.4 + 2.5 = 1

# Balance - In
# 1.2 + 2.2 + 3.2 + 4.2 + 5.2 = 1

# Flow Out

Flow_out = list(range(N))

for i in range(N): 
    Flow_out[i] = mymodel.Constraint(1, 1)
    for j in range(N):
        if i != j: Flow_out[i].SetCoefficient(use_arc[i][j], 1)
        
# Flow In

Flow_in = list(range(N))

for i in range(N):
    Flow_in[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        if i!= j: Flow_in[i].SetCoefficient(use_arc[j][i], 1)

### Position Constraints

In [11]:
# position 1 = 1
#position_1 = mymodel.Constraint(1, 1)
#position_1.SetCoefficient(pos[0], 1)

# position N = N
#position_N = mymodel.Constraint(1, 1)
#position_N.SetCoefficient(pos[N-1], N)

# MTZ Constraint

# u[i] - u[j] + (n-1)*x[i][j] <= (n-2)

MTZ = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 

for i in range(1,N):
    for j in range(1,N):
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[i][j].SetCoefficient(use_arc[i][j], N-1)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [12]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % shortest_route.Value())

# Display optimal solution
for i in range(N):
    print('pos[%d] = %d' % (i, pos[i].solution_value()))
    for j in range(N):
        print('From ', i, ' to ', j, ': ', use_arc[i][j].solution_value(), sep = '')
        
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 

Solution Status = 0
Number of variables = 20
Number of constraints = 17
Optimal Solution:
Optimal Value = 33.00
pos[0] = 0
From 0 to 0: 0.0
From 0 to 1: 1.0
From 0 to 2: 0.0
From 0 to 3: 0.0
pos[1] = 1
From 1 to 0: 0.0
From 1 to 1: 0.0
From 1 to 2: 1.0
From 1 to 3: 0.0
pos[2] = 2
From 2 to 0: 0.0
From 2 to 1: 0.0
From 2 to 2: 0.0
From 2 to 3: 1.0
pos[3] = 3
From 3 to 0: 1.0
From 3 to 1: 0.0
From 3 to 2: 0.0
From 3 to 3: 0.0
